### Estadisticas
#### 1 sample 1 epoch gpu 1
tarda de media 30 segundos.
#### 2 sample 1 epoch gpu 1
tarda de media 58 segundos.
#### 2 sample 1 epoch gpu 0.5
tarda de media 30 segundos. (y la gpu esta al 44% y le sobra memoria.)
#### 5 sample 1 epoch gpu 0.2
ocurren errores por memoria insuficiente en la ram (15.4 disponible para uso, del programa, y no alcanza.)
#### 4 sample 1 epoch gpu 0.25
tarda de media 1 minuto con 13 segundos, utiliza 11 gb de memoria ram, la gpu esta a mitad de capacidad de computo.
la cantidad de memoria ram es un cuello de botella para utilizar todo el poder de la GPU.
#### 4 sample 10 epoch gpu 0.25
tarda de media 2 minutos con 11 segundos, pero hay que tener cuidado ya que mientras entrena consume poca memoria, pero al momento de validar su loss y acc incrementa al nivel de warning con la RAM.
#### 3 sample 10 epoch gpu 0.33333
tarda 1 minuto y 40 segundos, pero con redes grandes tambien llega al warning de RAM.
#### 2 sample 10 epoch gpu 0.5
58 segundos, no se acerca a warning de RAM.
#### 10 sample 10 epoch gpu 0.5
4 minutos 25 segundos, corriendo de a dos modelos a la vez, maxima cantidad de memoria RAM usada 12.5 GB

2 configuraciones llegaron hasta las 10 epoch, 1 hasta la octava, el resto tuvo un desempeño pobre  desde el inicio y ray tune (programador ASHA) los termino en la primera epoch (se puede configurar la cantidad minima de epoch de ejecucion antes de terminarlas).

### Probar buscar hiperparametros de la red.
se deja estaticos el learning rate y el batch size. a 1-e4 y 16, respectivamente.

El resto se prueba buscar, para ver el tiempo y rendimiento se prueba 10 samples 50 epoch gpu 0.5

el resultado es una media de 14 minutos.


In [5]:
# python
import os
from functools import partial
# pytorch
import torch
from torch import nn
# pytorch utilidades
from torch.utils.data import random_split
# ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
# yo
from cargar_datos import cargar_datasets, cargar_dataloaders
from modelo_convolucional import instanciarRed


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0):
    data_dir = os.path.abspath("./data")
    checkpoint_dir = os.path.abspath("./checkpoint_dir")

    # se carga una vez, para que si no esta descargado, se descarge y se valide, el resto de veces en las pruebas, se saltara esta parte.
    cargar_datasets(data_dir)
    """
    variable de configuracion, Ray tune modificara sus valores en el proceso de ajuste de hiperparametros.
    """
    config = {
        #"batch_size": tune.grid_search([8, 16, 32]),
        # "learning_rate": tune.loguniform(1e-4, 1e-1),
        "batch_size": 16,
        "learning_rate": 1e-4,
        # red convolucional
        "cant_filtros_conv1": tune.grid_search([6, 12, 18]),
        "kernel_size_maxpool1": tune.grid_search([2, 3]),
        "cant_filtros_conv2": tune.grid_search([16, 22, 28]),
        "kernel_size_maxpool2": tune.grid_search([2, 3]),
        "full_l1": tune.grid_search([120, 140, 160]),
        "full_l2": tune.grid_search([84, 104, 124])
    }
    """
    configurar Ray y correr las pruebas en busqueda de hiperparametros
    """
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=list(config.keys()),
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        # partial es una funcion de orden superior de tipo curry
        partial(train_cifar, data_dir=data_dir, checkpoint_dir=checkpoint_dir, max_num_epochs=max_num_epochs),
        resources_per_trial={"cpu": 4, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    """
    Validar el desempeño de la red con mejor puntuacion en la busqueda de hiperparametros contra el conjunto de datos de validacion.
    """
    best_trained_model = instanciarRed(best_trial.config)
    device = getDevice()
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.dir_or_data
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device, data_dir)
    print("Best trial test set accuracy: {}".format(test_acc))


def test_accuracy(model, device="cpu", data_dir="./data"):
    _, testset = cargar_datasets(data_dir)
    testloader = cargar_dataloaders(testset, 4)

    correct = 0
    total = 0
    # pone a la red en moodo evaluacion, desactiva capas dropout.
    model.eval()
    # desactiva el proceso de calculo y guardado de valores intermedios
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


def getDevice():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            raise Exception(
                "la pc cuenta con multiples gpus, deberia utilizar DataParallel")
            #model = nn.DataParallel(model)
    return device


def train_cifar(config, checkpoint_dir=None, data_dir=None,max_num_epochs=10) :
    model = instanciarRed(config)

    device = getDevice()
    # le dice a Pytorch donde debe ejecutar el modelo.
    model.to(device)
    # CrossEntropyLoss requiere el output "logits", no es necesario pasarlo por el softmax, ya que lo calcula dentro. https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    criterion = nn.CrossEntropyLoss()

    # se necesita enviar los parametros del modelo al optimizador para que los pueda actualizar.
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["learning_rate"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, _ = cargar_datasets(data_dir)
    # reservar una parte de los datos de entrenamiento para validacion.
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = cargar_dataloaders(train_subset, config["batch_size"])
    valloader = cargar_dataloaders(val_subset, config["batch_size"])

    for epoch in range(max_num_epochs):  # loop over the dataset multiple times

        train_loop(model, device, trainloader, criterion, optimizer, epoch)
        loss, accuracy = validation_loss(model, device, valloader, criterion)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)
    print("Finished Training")


def train_loop(model, device, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_steps += 1
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0


def validation_loss(net, device, valloader, criterion):
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    net.eval()
    for i, data in enumerate(valloader, 0):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    loss = val_loss / val_steps
    accuracy = correct / total
    return loss, accuracy


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    """
    si config posee tune.grid_search, en alguno parametro, entonces num_samples contendra no la cantidad de ejemplos que le decis en num_samples, sino num_samples *(combinatoria grid search)
    ej: dos parametros grid search con 3 y 4 elementos, sus combinaciones son 3*4 = 12, si num_samples es 3 entonces es 12*3 = 36.
    """
    main(num_samples=1, max_num_epochs=50, gpus_per_trial=0.5)


== Status ==
Current time: 2022-11-19 21:10:18 (running for 03:09:26.77)
Memory usage on this node: 8.6/15.4 GiB 
Using AsyncHyperBand: num_stopped=266
Bracket: Iter 32.000: -0.9614356261372566 | Iter 16.000: -1.0854247627735139 | Iter 8.000: -1.2628516407966615 | Iter 4.000: -1.4272195230960847 | Iter 2.000: -1.567365212392807 | Iter 1.000: -1.7323409079551695
Resources requested: 8.0/20 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_18-00-51
Number of trials: 291/324 (1 ERROR, 22 PENDING, 2 RUNNING, 266 TERMINATED)
+-------------------------+------------+-----------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+---------+------------+----------------------+
| Trial name              | status     | loc                   |   batch_size |   learning_rate |   cant_filtr

74.6%(func pid=1138576) 
94.1%(func pid=1138801) 
74.6%(func pid=1138576) 
94.2%(func pid=1138801) 
74.6%(func pid=1138576) 
95.4%(func pid=1138801) 
74.7%(func pid=1138576) 
95.6%(func pid=1138801) 
74.9%(func pid=1138576) 
75.0%(func pid=1138576) 
75.1%(func pid=1138576) 
75.1%(func pid=1138576) 
75.7%(func pid=1138576) 
96.1%(func pid=1138801) 
96.1%(func pid=1138801) 
96.1%(func pid=1138801) 
76.6%(func pid=1138576) 
97.5%(func pid=1138801) 
76.6%(func pid=1138576) 
97.6%(func pid=1138801) 
76.7%(func pid=1138576) 
97.6%(func pid=1138801) 
76.7%(func pid=1138576) 
97.7%(func pid=1138801) 
97.7%(func pid=1138801) 
76.8%(func pid=1138576) 
98.3%(func pid=1138801) 
98.3%(func pid=1138801) 
98.4%(func pid=1138801) 
98.4%(func pid=1138801) 
98.5%(func pid=1138801) 
98.6%(func pid=1138801) 
98.7%(func pid=1138801) 
76.8%(func pid=1138576) 
98.7%(func pid=1138801) 
98.7%(func pid=1138801) 
77.8%(func pid=1138576) 
99.8%(func pid=1138801) 
78.0%(func pid=1138576) 
100.0%36m(func pid=113880

(func pid=1138801) Extracting ./data/cifar-10-python.tar.gz to ./data


80.0%(func pid=1138576) 
81.0%(func pid=1138576) 
82.1%(func pid=1138576) 
83.1%(func pid=1138576) 
84.2%(func pid=1138576) 


(func pid=1138801) Files already downloaded and verified


...

72.5%(func pid=1184962) 
73.4%(func pid=1184962) 
73.4%(func pid=1184962) 
74.2%(func pid=1184962) 
75.1%(func pid=1184962) 
75.9%(func pid=1184962) 
76.8%(func pid=1184962) 
77.7%(func pid=1184962) 
77.7%(func pid=1184962) 
78.6%(func pid=1184962) 
78.7%(func pid=1184962) 
79.4%(func pid=1184962) 
80.6%(func pid=1184962) 
80.6%(func pid=1184962) 
81.2%(func pid=1184962) 
82.4%(func pid=1184962) 
82.5%(func pid=1184962) 
82.5%(func pid=1184962) 
82.5%(func pid=1184962) 
83.2%(func pid=1184962) 
84.4%(func pid=1184962) 
85.1%(func pid=1184962) 
85.2%(func pid=1184962) 
86.2%(func pid=1184962) 
86.3%(func pid=1184962) 


== Status ==
Current time: 2022-11-19 21:24:43 (running for 03:23:51.91)
Memory usage on this node: 11.3/15.4 GiB 
Using AsyncHyperBand: num_stopped=321
Bracket: Iter 32.000: -0.9614356261372566 | Iter 16.000: -1.0854247627735139 | Iter 8.000: -1.2630366757392884 | Iter 4.000: -1.4273403955459594 | Iter 2.000: -1.569285293865204 | Iter 1.000: -1.740810968208313
Resources requested: 8.0/20 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_18-00-51
Number of trials: 324/324 (1 ERROR, 2 RUNNING, 321 TERMINATED)
+-------------------------+------------+-----------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+----------+------------+----------------------+
| Trial name              | status     | loc                   |   batch_size |   learning_rate |   cant_filtros_conv1 | 

87.1%(func pid=1184962) 
87.2%(func pid=1184962) 
88.1%(func pid=1184962) 
89.0%(func pid=1184962) 
89.8%(func pid=1184962) 
90.7%(func pid=1184962) 
91.6%(func pid=1184962) 
92.2%(func pid=1184962) 
92.2%(func pid=1184962) 
93.4%(func pid=1184962) 
94.0%(func pid=1184962) 
95.2%(func pid=1184962) 
95.7%(func pid=1184962) 
96.9%(func pid=1184962) 
97.0%(func pid=1184962) 
97.0%(func pid=1184962) 
97.1%(func pid=1184962) 
97.6%(func pid=1184962) 
98.7%(func pid=1184962) 
99.4%(func pid=1184962) 
100.0%36m(func pid=1184962) 


(func pid=1184962) Extracting ./data/cifar-10-python.tar.gz to ./data
(func pid=1184730) [1,  2000] loss: 1.961
(func pid=1184962) Files already downloaded and verified
(func pid=1184730) [2,  2000] loss: 1.687
(func pid=1184962) [1,  2000] loss: 1.978
== Status ==
Current time: 2022-11-19 21:24:53 (running for 03:24:01.70)
Memory usage on this node: 13.9/15.4 GiB 
Using AsyncHyperBand: num_stopped=321
Bracket: Iter 32.000: -0.9614356261372566 | Iter 16.000: -1.0854247627735139 | Iter 8.000: -1.2630366757392884 | Iter 4.000: -1.4273403955459594 | Iter 2.000: -1.569285293865204 | Iter 1.000: -1.740242818069458
Resources requested: 8.0/20 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_18-00-51
Number of trials: 324/324 (1 ERROR, 2 RUNNING, 321 TERMINATED)
+-------------------------+------------+-----------------------+--------------+-----------------+----------------------+-------------

TuneError: ('Trials did not complete', [train_cifar_40296_00189])